In [ ]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")

## Reference Trajectory

In [ ]:
"""
    reference_trajectory(model, times)

Return a reference trajectory that translates the walker from an x position of `xinit` to `xterm`,
with a nominal body height of `height` meters.
"""
function reference_trajectory(model::PlanarQuadruped, times, t_trans, xinit, xterm, uinit, init_mode)
    # Some useful variables
    n, m = size(model)
    tf = times[end]
    N = length(times)
    Δx = xterm - xinit
    dt = times[2]

    g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

    # initialization
    k_trans = 0
    for k = 1:N-1
        if times[k] < t_trans && times[k+1] >= t_trans
            k_trans = k + 1
            break
        end
    end

    Xref = zeros(n,N)
    Uref = zeros(m,N)

    Xref[:, 1] .= xinit
    Uref[:, 1] .= uinit

    Xref[:, 2:end] .= xterm

    a = 2 * Δx[7:10] / (t_trans^2)     # acceleration of feet (assume feet have constant acceleration and zero velocity since touching the ground)

    for k = 2:k_trans-1
        Uref[1:4, :] .= uinit[1:4] + a * (k-1)*dt  # v1 and v2
    end

    if init_mode == 1
        Uref[6, :] .= mb*g/2           # F1_y
        Uref[8, k_trans:end] .= mb*g/2    # F2_y
    else
        Uref[6, k_trans:end] .= mb*g/2    # F1_y
        Uref[8, :] .= mb*g/2           # F2_y
    end

    # Convert to a trajectory
    Xref = [SVector{n}(x) for x in eachcol(Xref)]
    Uref = [SVector{m}(u) for u in eachcol(Uref)]
    return Xref, Uref
end

## Problem Definition

In [ ]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 1.0
dt = 0.05
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.5

# Initial Conditions
# currently, we assume the initial mode ID is 1
xinit = [-0.4;0.6;3.1415/9; 0.0;-0.5;0.0; 0.0;0.0; -1.0;0.2]
xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0;0.0; 0.0;0.0; -lb;0.0]

uinit = [0.0;0.0; 0.0;-0.2; 0.0;mb*g/2; 0.0;mb*g/2]
init_mode = 1

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times, t_trans, xinit, xterm, uinit, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([10.0;10.0;1.0; 10.0;10.0;1.0; 10.0;10.0; 10.0;10.0])
R = Diagonal(fill(1e-3,8))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [ ]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

In [ ]:
# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-2, tol=1e-2)